In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
%matplotlib inline 
import seaborn as sns 
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score, KFold
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn import ensemble
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from matplotlib.legend_handler import HandlerLine2D
from sklearn.preprocessing import LabelEncoder

### data visualization

We will try to load our Training and Test data set with some Pandas functions as well as inspect it in order to get an idea of the data we're working with. It is necessary to understand the data features before to start running any model.

In [ ]:
train=pd.read_csv("../input/house-prices-advanced-regression-techniques/train.csv")
test=pd.read_csv("../input/house-prices-advanced-regression-techniques/test.csv")

In [ ]:
train.shape,test.shape 

The Training data contains 123 rows and 8 columns (features), testing data has 40 rows and 6 columns.

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train.dtypes 

In [ ]:
idn=test["Id"]

we will need that later for submission

In [ ]:
train.isnull().sum()

In [ ]:
train.info()

In [ ]:
train.keys()

## Inspecting the target

In [ ]:
train["SalePrice"].hist(bins=100)

We can clearly see that the target variable has a normal ditribution that is skewed towards the left. Now let's calculate the Skewness and Kurtosis :

#skewness & kurtosis

In [ ]:
print("the skewness of the target is %f "% train["SalePrice"].skew())
print('the kurtosis of the target is %f'% train["SalePrice"].kurt())

In fact skewed data can affect our machine learning's model so we will fix that by simply repalacing SalePrice by it's log 

In [ ]:
cormat=train.corr()
plt.subplots(figsize=(40,22))
sns.set(font_scale=1.45)
sns.heatmap(cormat,square=True,cmap='coolwarm')

In [ ]:
corrolation=cormat["SalePrice"].sort_values(ascending=False)
features=corrolation.index[:10]
features 

As we can see these are the features that are most related to our target , this will give us an idea of what features we will focus on them

In [ ]:
sns.pairplot(train[features],size=1.5)

In [ ]:
train.drop(['Id'],axis=1,inplace=True)

We drop the Id column because it does not add any usefull information to our model

In [ ]:
test.drop(['Id'],axis=1,inplace=True)

In [ ]:
train_df=train.copy()
test_df=test.copy()

 Now we must deal with the categorical features and concert them into numerical features but first we muissing values in these categorical features 

In [ ]:
train_cat=train.select_dtypes(include='object')

In [ ]:
train_cat.isna().sum()

In [ ]:
test_cat=test_df.select_dtypes(include='object')

In [ ]:
test_cat.isna().sum()

In [ ]:
def transform(train,cat):
    for i in cat.columns:
        train[i].fillna("None",inplace=True)
    return train    

In [ ]:
df_train=transform(train_df,train_cat)
df_test=transform(test_df,test_cat)

converting the categorical features to numerical features using Label encoder methode 

[https://miro.medium.com/max/588/1*KdCvKnI9ATVPiozmuRindA.png]

In [ ]:
cat=df_train.select_dtypes(include='object')

In [ ]:
from category_encoders import CountEncoder
enc = CountEncoder(normalize=True, cols=cat.columns)
df_train = enc.fit_transform(df_train)

In [ ]:
cat_test=df_test.select_dtypes(include='object')

In [ ]:
enct=CountEncoder(normalize=True,cols=cat_test.columns)
df_test=enct.fit_transform(df_test)

In [ ]:
df_test.info()

then we use Multivariate Imputation By Chained Equations (MICE) it s an excellent methode to fill the missing data this video will explain how the MICE works  
(https://www.youtube.com/watch?v=WPiYOS3qK70)

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [ ]:
estimator=RandomForestRegressor(max_depth=8)
mice = IterativeImputer(estimator=estimator,random_state=11,skip_complete=True)

In [ ]:
final_train=mice.fit_transform(df_train)
final_train=pd.DataFrame(final_train,columns=df_train.columns)

In [ ]:
final_test=mice.fit_transform(df_test)
final_test=pd.DataFrame(final_test,columns=df_test.columns)

In [ ]:
final_test.info()

In [ ]:
final_train["LogPrice"]=np.log(final_train["SalePrice"])

In [ ]:
final_train.head()

### Machine learning models 

In [ ]:
X_train = final_train.drop(["SalePrice","LogPrice"], axis=1)
y_train = final_train["LogPrice"]

## feature scaling 

first we use features scaling methode this will helps the machine to learn quicly and have a better accuracy 

In [ ]:
from sklearn.preprocessing import StandardScaler
scale=StandardScaler()
X_train=scale.fit_transform(X_train)

We split our data into trainnig data and validation data.IN fact the validation data gives us a good look about the performance of our model before testing it in the testing data and maybe avoiding overfiting



In [ ]:
X_training,X_valid,y_training,y_valid=train_test_split(X_train,y_train,test_size=0.2,random_state=0)

In [ ]:
lm=LinearRegression()
lm.fit(X_training,y_training)
print(lm)

Model Evaluation :Let's evaluate the model by checking out it's coefficients and how we can interpret them. This coefficient measures the intensity of the linear relationship between Y and X .For linear models, the intercept is the value of the linear predictor when all covariates are zero. In linear regression, this is equivalent to the y-intercept of the line of best fit. In logistic regression, it is the log odds of the baseline group

In [ ]:
print(lm.intercept_)

In [ ]:
print(lm.coef_)

In [ ]:
prediction=lm.predict(X_valid)
prediction=prediction.reshape(-1,1) 

In [ ]:
submission_pred=np.exp(prediction)

In [ ]:
print("MAE",metrics.mean_absolute_error(y_valid,submission_pred))
print("MSE",metrics.mean_squared_error(y_valid,submission_pred))
print("RMSE",np.sqrt(metrics.mean_squared_error(y_valid,submission_pred)))

MAE'is the mean absolute error it s simply the mean of all error it measures accuracy of continuis variables 'MSE' is great for ensuring that our trained model has no outlier predictions with huge errors 'RMSE'it s purpose is To evaluate trained models for usefulness / accuracy

In [ ]:
linreg=LinearRegression()
parameters_lin={"fit_intercept":[True,False],"normalize":[True,False],"copy_X":[True,False]}
grid_linreg=GridSearchCV(linreg,parameters_lin,verbose=1,scoring="r2")
grid_linreg.fit(X_training,y_training)
print("Best linreg Model"+str(grid_linreg.best_estimator_))
print("Best score :"+str(grid_linreg.best_score_))

# adding GridSearchfunction

Grid-search is used to find the optimal hyperparameters of a model which results in the most ‘accurate’ predictions

In [ ]:
linreg = grid_linreg.best_estimator_
linreg.fit(X_training, y_training)
lin_pred = linreg.predict(X_valid)
r2_lin = r2_score(y_valid, lin_pred)
rmse_lin = np.sqrt(mean_squared_error(y_valid, lin_pred))
print("R^2 Score: " + str(r2_lin))
print("RMSE Score: " + str(rmse_lin))

In [ ]:
scores_lin = cross_val_score(linreg, X_training, y_training, cv=10, scoring="r2")
print("Cross Validation Score: " + str(np.mean(scores_lin)))

In [ ]:
ridge = Ridge()
parameters_ridge = {"fit_intercept" : [True, False], "normalize" : [True, False], "copy_X" : [True, False], "solver" : ["auto"]}
grid_ridge = GridSearchCV(ridge, parameters_ridge, verbose=1, scoring="r2")
grid_ridge.fit(X_training, y_training)

print("Best Ridge Model: " + str(grid_ridge.best_estimator_))
print("Best Score: " + str(grid_ridge.best_score_))

In [ ]:
ridge = grid_ridge.best_estimator_
ridge.fit(X_training, y_training)
ridge_pred = ridge.predict(X_valid)
r2_ridge = r2_score(y_valid, ridge_pred)
rmse_ridge = np.sqrt(mean_squared_error(y_valid, ridge_pred))
print("R^2 Score: " + str(r2_ridge))
print("RMSE Score: " + str(rmse_ridge))

In [ ]:
scores_ridge = cross_val_score(ridge, X_training, y_training, cv=10, scoring="r2")
print("Cross Validation Score: " + str(np.mean(scores_ridge)))

## Decision Tree Model

In [ ]:
dtreg = DecisionTreeRegressor(random_state = 100)
parameters_dtr = {"criterion" : ["mse", "friedman_mse", "mae"], "splitter" : ["best", "random"], "min_samples_split" : [2, 3, 5, 10], 
                  "max_features" : ["auto", "log2"]}
grid_dtr = GridSearchCV(dtreg, parameters_dtr, verbose=1, scoring="r2")
grid_dtr.fit(X_training, y_training)

print("Best DecisionTreeRegressor Model: " + str(grid_dtr.best_estimator_))
print("Best Score: " + str(grid_dtr.best_score_))

In [ ]:
dtr = grid_dtr.best_estimator_
dtreg.fit(X_training, y_training)
dtr_pred = dtreg.predict(X_valid)
r2_dtr = r2_score(y_valid, dtr_pred)
rmse_dtr = np.sqrt(mean_squared_error(y_valid, dtr_pred))
print("R^2 Score: " + str(r2_dtr))
print("RMSE Score: " + str(rmse_dtr))

In [ ]:
scores_dtr = cross_val_score(dtreg, X_training, y_training, cv=10, scoring="r2")
print("Cross Validation Score: " + str(np.mean(scores_dtr)))

## Random forest 

In [ ]:
rfr = RandomForestRegressor()
paremeters_rf = {"n_estimators" : [5, 10, 15, 20], "criterion" : ["mse" , "mae"], "min_samples_split" : [2, 3, 5, 10], 
                 "max_features" : ["auto", "log2"]}
grid_rf = GridSearchCV(rfr, paremeters_rf, verbose=1, scoring="r2")
grid_rf.fit(X_training, y_training)

print("Best RandomForestRegressor Model: " + str(grid_rf.best_estimator_))
print("Best Score: " + str(grid_rf.best_score_))

In [ ]:
rf = grid_rf.best_estimator_
rfr.fit(X_training, y_training)
rf_pred = rfr.predict(X_valid)
r2_rf = r2_score(y_valid, rf_pred)
rmse_rf = np.sqrt(mean_squared_error(y_valid, rf_pred))
print("R^2 Score: " + str(r2_rf))
print("RMSE Score: " + str(rmse_rf))

In [ ]:
scores_rf = cross_val_score(rfr, X_training, y_training, cv=10, scoring="r2")
print("Cross Validation Score: " + str(np.mean(scores_rf)))

## Xgboot Model 

In [ ]:
xgboost = XGBRegressor(learning_rate=0.01,n_estimators=20000,
                                     max_depth=3, min_child_weight=0,
                                     gamma=0, subsample=0.7,
                                     colsample_bytree=0.7,
                                     objective='reg:linear', nthread=-1,
                                     scale_pos_weight=1, seed=27,
                                     reg_alpha=0.006)
xgb = xgboost.fit(X_training, y_training)

In [ ]:
xgb_pred = xgb.predict(X_valid)
r2_xgb = r2_score(y_valid, xgb_pred)
rmse_xgb = np.sqrt(mean_squared_error(y_valid, xgb_pred))
print("R^2 Score: " + str(r2_xgb))
print("RMSE Score: " + str(rmse_xgb))

## LGBM  regression 

In [ ]:
from lightgbm import LGBMRegressor

lightgbm = LGBMRegressor(objective='regression', 
                                       num_leaves=4,
                                       learning_rate=0.01, 
                                       n_estimators=20000,
                                       max_bin=2000, 
                                       bagging_fraction=0.75,
                                       bagging_freq=5, 
                                       bagging_seed=7,
                                       feature_fraction=0.2,
                                       feature_fraction_seed=7,
                                       verbose=-1,
                                       )
gbm = lightgbm.fit(X_training, y_training)

In [ ]:
gbm_pred = gbm.predict(X_valid)
r2_gbm = r2_score(y_valid, gbm_pred)
rmse_gbm = np.sqrt(mean_squared_error(y_valid, gbm_pred))
print("R^2 Score: " + str(r2_gbm))
print("RMSE Score: " + str(rmse_gbm))

In [ ]:
model_performances = pd.DataFrame({
    "Model" : ["Linear Regression", "Ridge", "Decision Tree Regressor", "Random Forest Regressor","XGBoost","LGBM Regressor"],
    "R Squared" : [str(r2_lin)[0:5], str(r2_ridge)[0:5],  str(r2_dtr)[0:5], str(r2_rf)[0:5],str(r2_xgb)[0:5], str(r2_gbm)[0:5]],
    "RMSE" : [str(rmse_lin)[0:8], str(rmse_ridge)[0:8],  str(rmse_dtr)[0:8], str(rmse_rf)[0:8],str(rmse_xgb)[0:8], str(rmse_gbm)[0:8]]
})
model_performances.round(4)

print("Sorted by R Squared:")
model_performances.sort_values(by="R Squared", ascending=False)

checking wich model is the best

In [ ]:
print("Sorted by RMSE:")
model_performances.sort_values(by="RMSE", ascending=True)

as we can see the XGBoost gave us the best score 

In [ ]:
def blend_models_predict(X):
    return ((0.05 * lm.predict(X)) + \
            (0.05 * linreg.predict(X)) + \
            (0.05 * ridge.predict(X)) + \
            (0.2 * gbm.predict(X)) + \
            (0.15 * rfr.predict(X)) + \
            (0.4 * xgb.predict(X)))

In [ ]:
submission_pred = np.exp(blend_models_predict(final_test))
print(submission_pred)

In [ ]:
submission_pred.shape

In [ ]:
submission_df=pd.DataFrame({"Id":idn,"SalePrice":submission_pred})

In [ ]:
submission_df.to_csv("houseprices222.csv", index=False)